# Chapter 10 신경망 기계번역

## 10.1 기계번역

- seq2seq
- 어텐션

### 10.1.1 번역의 목표

$$\hat{e} = argmax P_{f \rightarrow e} (e|f)$$

#### 번역이 어려운 이유
- 언어의 모호성
- 문화의 차이

### 10.1.2 기계번역의 역사

#### 규칙 기반 기계번역(RBMT)

- 구조를 분석해서 얻은 규칙을 통해서 번역
- 한계
    - 규칙을 사람이 일일이 찾아야해서 많은 자원과 시간 소모
    - 번역 언어쌍을 확장할 때 매우 불리
    
#### 통게 기반 기계번역(SMT)

- 대량의 양방향 코퍼스에서 통계를 얻어내 번역 시스템 구성
- 언어쌍을 확장할 때 대부분의 알고리즘과 시스템은 유지되서 RBMT보다 유리

#### 딥러닝 이전의 신경망 기계번역

- 오토인코더 기반: 인코더 $\rightarrow$ 디코더
- 성능 매우 떨어짐

#### 딥러닝 이후의 신경망 기계번역

장점
1. end-to-end 모델  
    여러 모듈로 복잡하게 나눠지지 않고 하나의 모델로 번역을 해서 성능 극대화
2. 더 나은 언어 모델  
    n-gram의 단점이었던 희소성 문제를 NNLM으로 해결, 자연스러운 번역 결과 생성 강점
3. 훌륭한 문장 임베딩  
    임베딩 벡터 만들어내는 능력이 뛰어나기 때문에 노이즈나 희소성 문제 더 잘 대처

## 10.2 seq2seq

### 10.2.1 구조 소개

$P(Y|X;\theta)$를 최대로 하는 모델 파라미터를 찾고 사후 확률을 최대로 하는 $Y$를 찾는다.

$$\hat{\theta} = \argmax_\theta P(Y|X;\theta) \ where \ X = \{x_1, x_2, \dots, x_n\}, Y = \{y_1, y_2, \dots, y_m\}$$
$$\hat{Y} = \argmax_{Y \in \mathcal{Y}}P(Y|X;\theta)$$

seq2seq는 **인코더**, **디코더**, **생성자**로 구성

#### 인코더

- 소스 문장으로부터 문장 임베딩 벡터를 만들어낸 후 $P(z|X)$를 모델링
- 주어진 문장을 매니폴드를 따라 차원 축소하여 해당 도메인의 잠재 공간의 어떤 하나의 점에 투영하는 작업
- 기계번역을 위한 문장 임베딩 벡터를 생성하려면 최대한 많은 정보를 갖고 있어야 함.

▶ 인코더 수식

$$h_t^{src} = RNN_{enc}(emb_{src}(x_t), h_{t-1}^{src})$$
$$H^{src} = [h_1^{src};h_2^{src};\dots;h_n^{src}]$$

▶ 전체 time-step 병렬 처리했을 때

$$H^{src} = RNN_{src}(emb_{src}(X), h_0^{src})$$

#### 디코더

조건부 신경망 언어 모델(CNNLM)

▶ seq2seq 모델 수식 time-step에 관해 풀었을 때
$$P_{\theta}(Y|X) = \prod_{t=1}^m P_{\theta}(y_t|X,y_{<t})$$
$$logP_{\theta}(Y|X) = logP_{\theta}(y_t|X,y_{<t})$$

- RNNLM에서 조건부 확률 변수 부분에 X가 추가된 모습
- 인코더의 결과인 문장 임베딩 벡터와 이전 time-step까지 번역하여 생성한 단어들에 기반하여 현재 time-step의 단어 생성

▶ 디코더 수식
$$h_t^{tgt} = RNN_{dec}(emb_{tgt}(y_{t-1}), h_{t-1}^{tgt})$$
$$where \ h_0^{tgt} = h_n^{tgt} and y_0 = BOS$$

#### 생성자

디코더에서 각 time-step별로 결과 벡터 $h_t^{tgt}$를 받아 softmax를 계산하여, 각 타깃 언어의 단어별 확률값을 반환

▶ 생성자 수식

$$\hat{y}_t = softmax(linear_{hs \to |V_{tgt}|}(h_t^{tgt})) and \hat{y}_m = EOS$$
$$where \ hs \ is \ hidden \ size \ of \ RNN, \ and \ |V_{tgt}| \ is \ size \ of \ output \ vocabulary$$

### 10.2.2. seq2seq의 활용 분야

기계번역, 챗봇, 요약, 기타 자연어 처리, 음성 인식, 독순술, 이미지 캡셔닝 등

### 10.2.3 한계점

#### 장기 기억력

정보를 압축하는데 한계가 있다보니까 시퀀스가 길어질수록 성능이 떨어진다.

#### 구조 정보의 부재

현재는 문장을 단순히 시퀀스 데이터로 다루고 있지만, 나중에는 구조 정보도 필요할 것이다.

#### 챗봇 또는 QA봇

번역이나 요약 문제의 경우에는 새롭게 추가되는 정보가 거의 없어서 잘 수행했지만, 대화의 경우에는 지식이나 맥락 등 추가되는 정보가 있기 때문에 한계가 있다.

### 10.2.4. 파이토치 예제 코드

#### 인코더 클래스

In [3]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    
    def __init__(self, word_vec_dim, hidden_size, n_layers=4, dropout_p=.2):
        super(Encoder, self).__init__()
        
        self.rnn = nn.LSTM(word_vec_dim,
                           int(hidden_size / 2),
                           num_layers=n_layers,
                           dropout=dropout_p,
                           bidirectional=True,
                           batch_first=True
                           )
        
    def forward(self, emb):
        if isinstance(emb, tuple):
            x, lengths = emb
            x = pack(x, lenghts.tolist(), batch_first=True)
        
        else:
            x = emb
            
        y, h = self.rnn(x)
    
        if isinstance(emb, tuple):
            y, _ = unpack(y, batch_first=True)
        
        return y, h

####  pack_padded_sequence 함수

In [13]:
a = [torch.tensor([1, 2, 3]), torch.tensor([3, 4]), torch.tensor([1])]
b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)

In [14]:
a

[tensor([1, 2, 3]), tensor([3, 4]), tensor([1])]

In [15]:
b

tensor([[1, 2, 3],
        [3, 4, 0],
        [1, 0, 0]])

In [20]:
torch.nn.utils.rnn.pack_padded_sequence(b, batch_first=True, lengths=[3, 2, 1])

PackedSequence(data=tensor([1, 3, 1, 2, 4, 3]), batch_sizes=tensor([3, 2, 1]), sorted_indices=None, unsorted_indices=None)

#### 디코더 클래스

어텐션 이후에

#### 생성자 클래스

In [21]:
class Generator(nn.Module):
    
    def __init__(self, hidden_size, output_size):
        super(Generator, self).__init__()
        
        self.output = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self, x):
        y = self.softmax(self.output(x))
        
        return y

#### 전체 seq2seq 클래스

어텐션 이후

#### 손실 함수

교차 엔트로피 함수

실제 구현에서는 **softmax + 교차 엔트로피**보다 **logsoftmax + 음의 로그 가능도(NLL)**를 사용한다.

In [ ]:
loss_weight = torch.ones(output_size)
loss_weight[data_loader.PAD] = 0.

crit = nn.NLLLoss(weight=loss_weight,
                  reduction='sum',
                  )

In [23]:
def _get_loss(self, y_hat, y, crit=None):
    crit = self.crit if crit is None else crit
    loss = crit(y_hat.contiguous().view(-1, y_hat.size(-1)),
                y.contiguous().view(-1))
    
    return loss

```contiguous()```: 특정 축 잘라내는 함수

In [24]:
x = torch.arange(1*2*2*6).view(-1,2,2,6)
print(x)

tensor([[[[ 0,  1,  2,  3,  4,  5],
          [ 6,  7,  8,  9, 10, 11]],

         [[12, 13, 14, 15, 16, 17],
          [18, 19, 20, 21, 22, 23]]]])


In [25]:
x = x.view(-1,6)
print(x)

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]])


In [27]:
x = x[:,:4].contiguous()
print(x)

tensor([[ 0,  1,  2,  3],
        [ 6,  7,  8,  9],
        [12, 13, 14, 15],
        [18, 19, 20, 21]])


## 10.3 어텐션

### 10.3.1 어텐션 이해하기

쿼리와 비슷한 값을 가진 키를 찾아서 값을 얻는 과정

### 10.3.2 key-value 함수

In [28]:
dic = {'computer': 9, 'dog': 2, 'cat': 3}

In [30]:
def key_value_func(query):
    weights = []
    
    for key in dic.keys():
        weights += [is_same(key, query)]
        
    weight_sum = sum(weights)
    for i,w in enumerate(weights):
        weights[i] = weights[i] / weight_sum
        
    answer = 0
    
    for weight, value in zip(weights, dic.values()):
        answer += weight *  value
        
    return answer

In [31]:
def is_same(key, query):
    if key == query:
        return 1.
    else:
        return 0.

In [35]:
for k in dic.keys():
    print(is_same(k, "computer"))

1.0
0.0
0.0


In [36]:
print(key_value_func("computer"))

9.0


### 10.3.3 연속적인 key-value 함수

...

### 10.3.4 연속적인 key-value 벡터 함수

...

### 10.3.5 기계번역에서의 어텐션

인코더의 각 time-step별 출력을 키와 밸류로 삼고, 현재 time-step의 디코더 출력을 쿼리로 삼아 어텐션 계산

▶ 어텐션을 추가한 seq2seq 수식

$$w = softmax(h_t^{tgtT}W \cdot H^{src})$$
$$c = H^{src} \cdot w \  and c \ is \ a\ context \ vector$$
$$\tilde{h}_t^{tgt} = tanh(linear_{2 \times hs \to hs}([h_t^{tgt};c]))$$
$$\hat{y}_t = softmax(linear_{hs \to |V_{tgt}|}(\tilde{h}_t^{tgt}))$$
$$where \ hs \ is \ hidden \ size \ of \ RNN, \ and \ |V_{tgt}| \ is \ size \ of \ output \ vocabulary$$

- 원하는 정보를 어텐션을 통해 인코더에서 획득한 후, 해당 정보를 디코더의 출력과 이어붙여 tanh를 취한 후, softmax 계산을 통해 다음 time-step의 입력이 되는 $\hat y_t$을 구한다.

<img src="https://blobscdn.gitbook.com/v0/b/gitbook-28427.appspot.com/o/assets%2F-LFzjkZt6ljMBd4YGVCn%2F-LX4NCwvtTXyg_vzXhF0%2F-LX4NFi6LreOTrZ3dcWF%2F10-03-01.png?generation=1548425967953999&alt=media" width="80%">

#### 선형 변환

소스 언어와 대상 언어가 애초부터 다르기 때문에 선형 관계가 있다고 가정하고 내적을 수행하기 전에 선형 변환을 해준다. $W$는 선형 변환을 위한 신경망 가중치 파라미터이다.

어텐션이 필요한 이유는 문장이 길어질수록 은닉 상태만으로 모든 정보를 완벽하게 전달할 수 없는 문제 때문이다. 따라서 디코더의 time-step마다 현재 디코더의 은닉 상태에 따라 필요한 인코더의 정보에 접근해서 사용하겠다는 것이다.

선형 변환을 배우는 것 자체를 어텐션이라고 표현할 수 있다. 이는 디코더의 현재 상태에 따라 필요한 쿼리를 만들어내고, 인코더의 key 값들과 비교해서 가중합을 하는 과정이기 때문이다.

선형 변환을 위한 가중치 파라미터가 훈련되면, 디코더의 상태 자체도 선형 변환에 좋은 방향으로 RNN이 동작할 것이다.

#### 선형 변환 결과

어텐션을 적용한 것이 성능이 더 좋았고, 특히 문장이 길어질수록 성능이 떨어지는 점이 덜해졌다.

### 10.3.6 파이토치 예제 코드

```torch.bmm()```: 배치 행렬 곱을 수행하는 함수, 뒤의 두개의 차원에 대해 행렬 곱을 수행하기 때문에 이 부분의 차원은 행렬 곱에 적합한 크기여야 하고, 앞의 차원들은 배치로 취급해서 같은 크기여야 한다.

In [39]:
import torch

x = torch.arange(80).view(10, 2, 4)
y = torch.arange(240).view(10, 4, 6)

x.size(), y.size()

(torch.Size([10, 2, 4]), torch.Size([10, 4, 6]))

In [40]:
# |x| = (10, 2, 4)
# |y| = (10, 4, 6)

z = torch.bmm(x, y)
# |z| = (10, 2, 6)

In [42]:
z.size()

torch.Size([10, 2, 6])

#### 어텐션 클래스

선형 변환을 위한 가중치 파라미터를 편향이 없는 선형 계층으로 대체

In [43]:
class Attention(nn.Module):
    
    def __init(self, hidden_size):
        super(Attention, self).__init__()
        
        self.linear = nn.Linear(hidden_size, hidden_size, bias=False)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, h_src, h_t_tgt, mask=None):
        query = self.linear(h_t_tgt.squeeze(1)).unsqueeze(-1)
        
        weight = torch.bmm(h_src, query).squeeze(-1)
        
        if mask is not None:
            weight.masked_fill_(mask, -float('inf'))
        weight = self.softmax(weight)
        
        context_vector = torch.bmm(weight.unsqueeze(1), h_src)
        
        return context_vector